In [1]:
# Andrew Kan, Govind Pillai

In [207]:
# Initialization
import pandas as pd
from nltk.stem import WordNetLemmatizer
import string
lemmatizer = WordNetLemmatizer()


translator = str.maketrans('', '', string.punctuation)

# Read in data, drop unimportant columns and rename column names to be more descriptive

emails_df = pd.read_csv("./data/spam.csv", encoding= "ISO-8859-1")
emails_df = emails_df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
emails_df.columns = ['type', 'body']

emails_smalldf = emails_df.head(100)

stop_words = ["and"]



In [208]:
word_dict = {}
for i, email in emails_smalldf.iterrows():
    for word in email['body'].split():
        # Remove punctuation and lemmatize
        cleaned_word = word.translate(translator)
        cleaned_word = lemmatizer.lemmatize(cleaned_word)
        if word in word_dict.keys(): 
            word_dict[cleaned_word] += 1
        else: 
            word_dict[cleaned_word] = 1

# Remove stop words      

for key in list(word_dict):
    if key in stop_words:
        del word_dict[key]
word_dict




{'': 1,
 '07732584351': 1,
 '0800': 1,
 '08000930705': 1,
 '08002986030': 1,
 '08452810075over18s': 1,
 '09061209465': 1,
 '09061701461': 1,
 '09066364589': 1,
 '1': 1,
 '100': 1,
 '10am9pm': 1,
 '11': 1,
 '12': 1,
 '150pday': 1,
 '150pm': 1,
 '16': 1,
 '169': 1,
 '186\x8c£150moreFrmMob': 1,
 '2': 3,
 '20000': 1,
 '2005': 1,
 '21st': 1,
 '2nd': 1,
 '3': 2,
 '3AJ': 1,
 '4': 2,
 '4403LDNW1A7RW18': 1,
 '4txtí120': 1,
 '530': 1,
 '6031': 1,
 '6days': 1,
 '81010': 1,
 '85069': 1,
 '87077': 2,
 '87121': 2,
 '87575': 1,
 '8am': 1,
 '9': 1,
 'A': 2,
 'ARE': 1,
 'AT': 1,
 'Abiola': 1,
 'AcoEntry41': 1,
 'Aft': 1,
 'Ahhh': 1,
 'Alright': 1,
 'Anything': 1,
 'Arabian': 1,
 'Ard': 1,
 'Are': 2,
 'As': 3,
 'B': 1,
 'BABE': 1,
 'Barbie': 1,
 'Bonus': 1,
 'Britney': 1,
 'Burns': 1,
 'But': 2,
 'C': 1,
 'CASH': 1,
 'CLAIM': 1,
 'CSH11': 1,
 'Call': 2,
 'Callers': 1,
 'Callertune': 1,
 'Claim': 1,
 'Co': 1,
 'Congrats': 1,
 'Correct': 1,
 'Cost': 1,
 'Cup': 1,
 'DATE': 1,
 'DIS': 1,
 'DO': 1,
 'Detroit